In [1]:
import wandb
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
import tempfile
import os
import plotly.express as px

In [2]:
import wandb
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tempfile
import os
import plotly.express as px

In [3]:
import wandb
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tempfile
import os
import plotly.express as px

In [4]:
# Login to Weights & Biases
wandb.login(relogin=True)

True

In [5]:
from scipy import stats
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.stats import pointbiserialr

In [6]:
#CREATING PROJETCT ON WANDB
df= pd.read_csv("listings.csv.gz")
df['price'] = pd.to_numeric(df['price'].str.replace(",","").str.strip('$'))
df.to_csv("raw_data.csv",index=False)
!wandb artifact put \
      --name mlops-final-project/raw_data.csv \
      --type raw_data \
      --description "The raw data from of airbnb" raw_data.csv

In [7]:
# save_code tracking all changes of the notebook and sync with Wandb
run = wandb.init(project="mlops-final-project", save_code=True)
# donwload the latest version of artifact raw_data.csv
artifact = run.use_artifact("mlops-final-project/raw_data.csv:latest")
# create a dataframe from the artifact
df = pd.read_csv(artifact.file())

In [8]:
df.head(2)

      id                         listing_url       scrape_id last_scraped  \
0  17878  https://www.airbnb.com/rooms/17878  20211224070558   2021-12-25   
1  24480  https://www.airbnb.com/rooms/24480  20211224070558   2021-12-25   

                                                name  \
0  Very Nice 2Br in Copacabana w. balcony, fast WiFi   
1   Nice and cozy near Ipanema Beach, w/ home office   

                                         description  \
0  Discounts for long term stays. <br />- Large b...   
1  My studio is located in the best of Ipanema, t...   

                               neighborhood_overview  \
0  This is the one of the bests spots in Rio. Bec...   
1  The beach, the lagoon, Ipanema is a great loca...   

                                         picture_url  host_id  \
0  https://a0.muscache.com/pictures/65320518/3069...    68997   
1  https://a0.muscache.com/pictures/11955612/b28e...    99249   

                                  host_url  ... review_scores_com

In [9]:
cardCols = []
for col in df.columns:
    num = df[col].nunique()/df.shape[0]
    if num > 0.1:
        cardCols.append(col)
df[cardCols].head(2)

      id                         listing_url  \
0  17878  https://www.airbnb.com/rooms/17878   
1  24480  https://www.airbnb.com/rooms/24480   

                                                name  \
0  Very Nice 2Br in Copacabana w. balcony, fast WiFi   
1   Nice and cozy near Ipanema Beach, w/ home office   

                                         description  \
0  Discounts for long term stays. <br />- Large b...   
1  My studio is located in the best of Ipanema, t...   

                               neighborhood_overview  \
0  This is the one of the bests spots in Rio. Bec...   
1  The beach, the lagoon, Ipanema is a great loca...   

                                         picture_url  host_id  \
0  https://a0.muscache.com/pictures/65320518/3069...    68997   
1  https://a0.muscache.com/pictures/11955612/b28e...    99249   

                                  host_url host_name  host_since  \
0  https://www.airbnb.com/users/show/68997  Matthias  2010-01-08   
1  https://www.a

In [10]:
## Dessas colunas as únicas de nosso interesse são  host_since,latitude,longitude,first_review
[cardCols.remove(x) for x in ['host_since','latitude','longitude','first_review','amenities']]
df = df.drop(cardCols, axis=1)
df.head(2)

        scrape_id last_scraped  host_since  \
0  20211224070558   2021-12-25  2010-01-08   
1  20211224070558   2021-12-25  2010-03-26   

                                     host_location  host_response_time  \
0  Rio de Janeiro, State of Rio de Janeiro, Brazil      within an hour   
1           Rio de Janeiro, Rio de Janeiro, Brazil  a few days or more   

  host_response_rate host_acceptance_rate host_is_superhost  \
0               100%                  96%                 f   
1               100%                 100%                 f   

  host_neighbourhood  host_listings_count  ...  review_scores_communication  \
0         Copacabana                  2.0  ...                          4.9   
1            Ipanema                  0.0  ...                          4.9   

  review_scores_location review_scores_value license instant_bookable  \
0                   4.76                4.66     NaN                f   
1                   4.97                4.58     NaN            

In [11]:
missing = (df.isna().sum()/len(df)).sort_values(ascending=False)
missing[:20]

license                         1.000000
neighbourhood_group_cleansed    1.000000
calendar_updated                1.000000
bathrooms                       1.000000
host_neighbourhood              0.479897
neighbourhood                   0.461811
review_scores_location          0.380341
review_scores_accuracy          0.380301
review_scores_cleanliness       0.380301
review_scores_checkin           0.380301
review_scores_communication     0.380260
review_scores_value             0.380260
first_review                    0.365025
last_review                     0.365025
review_scores_rating            0.365025
reviews_per_month               0.365025
host_acceptance_rate            0.230030
host_response_time              0.158092
host_response_rate              0.158092
bedrooms                        0.060695
dtype: float64

In [12]:
df = df.drop(missing[missing > 0.35].index, axis=1)
df.head(2)

        scrape_id last_scraped  host_since  \
0  20211224070558   2021-12-25  2010-01-08   
1  20211224070558   2021-12-25  2010-03-26   

                                     host_location  host_response_time  \
0  Rio de Janeiro, State of Rio de Janeiro, Brazil      within an hour   
1           Rio de Janeiro, Rio de Janeiro, Brazil  a few days or more   

  host_response_rate host_acceptance_rate host_is_superhost  \
0               100%                  96%                 f   
1               100%                 100%                 f   

   host_listings_count  host_total_listings_count  ... availability_365  \
0                  2.0                        2.0  ...              257   
1                  0.0                        0.0  ...              107   

  calendar_last_scraped number_of_reviews number_of_reviews_ltm  \
0            2021-12-25               267                    10   
1            2021-12-25                85                     0   

   number_of_reviews

In [13]:
#Drop Useless
df = df.drop(['scrape_id','last_scraped','host_location','calendar_last_scraped','host_verifications','property_type'], axis=1)

In [14]:
missing = (df.isna().sum()/len(df)).sort_values(ascending=False)
missing[missing.values > 0]

host_acceptance_rate         0.230030
host_response_rate           0.158092
host_response_time           0.158092
bedrooms                     0.060695
beds                         0.014135
bathrooms_text               0.002607
host_has_profile_pic         0.000611
host_since                   0.000611
host_total_listings_count    0.000611
host_listings_count          0.000611
host_is_superhost            0.000611
host_identity_verified       0.000611
dtype: float64

In [15]:
#%%capture
# Changin host since to month count
df['host_since'] =  pd.to_datetime(df['host_since'], format='%Y-%m-%d')
def diff_month(x):
    curr_time = pd.to_datetime("now")
    return (x.year - curr_time.year) * 12 + x.month - curr_time.month
df['host_since'] = df['host_since'].apply(lambda x: diff_month(x))

In [16]:
#Changing bathrooms text to numeric
df.loc[df['bathrooms_text'].str.contains('half', na=False), 'bathrooms_text'] = '0.5'
df['bathrooms_text'] = pd.to_numeric(df['bathrooms_text'].str.replace("[a-zA-Z\s-]",""))

In [17]:
## Removing Percentege
for colName in ['host_response_rate','host_acceptance_rate']:
    df[colName ] = df[colName ].str.rstrip('%').astype('float')/100.0

In [18]:
## Changin Categorical NA to mode
for colName in ['host_has_profile_pic','host_is_superhost','host_identity_verified','host_response_time','host_since']:
    df[colName ] = df[colName].fillna(df[colName].mode()[0])
for colName in ['host_acceptance_rate','host_response_rate','bedrooms','beds',
                'bathrooms_text','host_total_listings_count','host_listings_count']:
    df[colName ] = df[colName].fillna(round(df[colName].mean(),2))

In [19]:
missing = (df.isna().sum()/len(df)).sort_values(ascending=False)
missing[missing.values > 0]

Series([], dtype: float64)

In [20]:
df['host_response_time']= df['host_response_time'].replace(['within an hour','within a few hours', 'within a day', 'a few days or more'],
                        [1, 2,3,4])
df['room_type']= df['room_type'].replace(['Shared room', 'Private room',  'Hotel room','Entire home/apt'],
                        [1, 2,3,4])
for colName in ['host_is_superhost','host_has_profile_pic','host_identity_verified','has_availability','instant_bookable']:
    df[colName] = df[colName].replace(['f','t'],
                            [0,1])

In [21]:
district= pd.get_dummies(df['neighbourhood_cleansed'])
if 'dfdistrict' in locals() : del dfdistrict
y = df['price']
for colName in district.columns:
    #calculate point-biserial correlation
    x = district[colName]
    corr, pVal = pointbiserialr(x, y)
    dfTemp = {'column': colName, 
            'correlation': corr, 
            'pVal': pVal}
    if 'dfdistrict' in locals():
        dfdistrict = dfdistrict.append(dfTemp, ignore_index = True)
    else:
        dfdistrict=pd.DataFrame(dfTemp, index=[0])
importantDistrict = dfdistrict.loc[dfdistrict['pVal'] < 0.5].sort_values(by='correlation', ascending=False)['column']
districs = district[importantDistrict]
districs.head(2)

   São Cristóvão  Joá  São Conrado  Ipanema  Leblon  Santa Cruz  \
0              0    0            0        0       0           0   
1              0    0            0        1       0           0   

   Barra da Tijuca  Alto da Boa Vista  Lagoa  Itanhangá  ...  Laranjeiras  \
0                0                  0      0          0  ...            0   
1                0                  0      0          0  ...            0   

   Camorim  Leme  Catete  Santa Teresa  Jacarepaguá  Tijuca  Flamengo  Centro  \
0        0     0       0             0            0       0         0       0   
1        0     0       0             0            0       0         0       0   

   Botafogo  
0         0  
1         0  

[2 rows x 37 columns]

In [22]:
listValues = df['amenities'].apply(lambda x: x.strip('][').split(', '))
def myfunc(a):
  return a.replace('"','')
df['amenities'] = listValues.apply(lambda x: list(map(myfunc, x)))

mlb = MultiLabelBinarizer(sparse_output=True)

dfAmenities = df['price'].to_frame().join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(df.pop('amenities')),
                index=df.index,
                columns=mlb.classes_))

In [23]:
if 'dfBi' in locals() : del dfBi 
    
y = dfAmenities['price']
for colName in dfAmenities.columns[1:]:
    #calculate point-biserial correlation
    x = dfAmenities[colName]
    corr, pVal = pointbiserialr(x, y)
    dfTemp = {'column': colName, 
            'correlation': corr, 
            'pVal': pVal}
    if 'dfBi' in locals():
        dfBi = dfBi.append(dfTemp, ignore_index = True)
    else:
        dfBi=pd.DataFrame(dfTemp, index=[0])
        
dfBi = dfBi.loc[dfBi['pVal'] < 0.5].sort_values(by='correlation', ascending=False)
importantCols = dfBi['column']
dfAmenities = dfAmenities[dfBi['column'].values]

In [24]:
dfAmenities.head(2)

   Hot tub  Pool  Free parking on premises  Air conditioning  Beachfront  \
0        1     0                         0                 1           0   
1        0     0                         0                 1           0   

   Dishwasher  Bathtub  Dryer  Crib  TV with standard cable  ...  \
0           0        0      0     0                       1  ...   
1           0        0      0     0                       1  ...   

   Cleaning products  Host greets you  Extra pillows and blankets  \
0                  0                0                           0   
1                  1                0                           1   

   Luggage dropoff allowed  Ceiling fan  Cooking basics  \
0                        1            0               1   
1                        0            1               0   

   Dishes and silverware  Essentials  Hot water  Lock on bedroom door  
0                      1           1          1                     0  
1                      1           1

In [25]:
binary= df[['host_is_superhost','host_has_profile_pic','host_identity_verified','has_availability','instant_bookable']]
y = df['price']
if 'dfbinary' in locals() : del dfbinary 
for colName in binary.columns:
    #calculate point-biserial correlation
    x = binary[colName]
    corr, pVal = pointbiserialr(x, y)
    dfTemp = {'column': colName, 
            'correlation': corr, 
            'pVal': pVal}
    if 'dfbinary' in locals():
        dfbinary = dfbinary.append(dfTemp, ignore_index = True)
    else:
        dfbinary=pd.DataFrame(dfTemp, index=[0])
importantbinary = dfbinary.loc[dfbinary['pVal'] < 0.5].sort_values(by='correlation', ascending=False)['column']
binary = binary[importantbinary]
binary

       has_availability
0                     1
1                     1
2                     1
3                     1
4                     1
...                 ...
24544                 1
24545                 1
24546                 1
24547                 1
24548                 1

[24549 rows x 1 columns]

In [26]:
binary = binary.join(dfAmenities).join(districs)
binary

       has_availability  Hot tub  Pool  Free parking on premises  \
0                     1        1     0                         0   
1                     1        0     0                         0   
2                     1        0     0                         0   
3                     1        0     0                         0   
4                     1        0     0                         0   
...                 ...      ...   ...                       ...   
24544                 1        0     0                         1   
24545                 1        0     0                         0   
24546                 1        0     0                         0   
24547                 1        0     0                         0   
24548                 1        0     0                         0   

       Air conditioning  Beachfront  Dishwasher  Bathtub  Dryer  Crib  ...  \
0                     1           0           0        0      0     0  ...   
1                     1    

In [27]:
notOrdinal = ['host_is_superhost','host_has_profile_pic','host_identity_verified','has_availability','instant_bookable',
              'neighbourhood_cleansed','latitude','longitude']
df = df.drop(columns=notOrdinal)
df

       host_since  host_response_time  host_response_rate  \
0          -150.0                   1                1.00   
1          -148.0                   4                1.00   
2          -147.0                   3                0.80   
3          -145.0                   1                1.00   
4          -145.0                   1                1.00   
...           ...                 ...                 ...   
24544       -50.0                   1                0.79   
24545       -95.0                   1                1.00   
24546       -74.0                   1                0.79   
24547       -86.0                   2                1.00   
24548      -124.0                   2                0.91   

       host_acceptance_rate  host_listings_count  host_total_listings_count  \
0                      0.96                  2.0                        2.0   
1                      1.00                  0.0                        0.0   
2                      0.41   

In [28]:
#ProfileReport(df, title="Pandas Profiling Report").to_file("report.html")

In [29]:
df = df.drop(['maximum_nights_avg_ntm','maximum_maximum_nights','minimum_maximum_nights','maximum_minimum_nights',
        'minimum_minimum_nights','maximum_nights','minimum_nights','availability_30','availability_60',
         'calculated_host_listings_count_private_rooms'], axis=1)

In [30]:
df = df.join(binary)

In [31]:
df

       host_since  host_response_time  host_response_rate  \
0          -150.0                   1                1.00   
1          -148.0                   4                1.00   
2          -147.0                   3                0.80   
3          -145.0                   1                1.00   
4          -145.0                   1                1.00   
...           ...                 ...                 ...   
24544       -50.0                   1                0.79   
24545       -95.0                   1                1.00   
24546       -74.0                   1                0.79   
24547       -86.0                   2                1.00   
24548      -124.0                   2                0.91   

       host_acceptance_rate  host_listings_count  host_total_listings_count  \
0                      0.96                  2.0                        2.0   
1                      1.00                  0.0                        0.0   
2                      0.41   

In [32]:
# Save the artifacts. We use a temporary directory so we do not leave
# any trace behind

with tempfile.TemporaryDirectory() as tmp_dir:
        # Make the artifact name from the provided root plus the name of the split
        artifact_name = "preprocessed_data.csv"
        # Get the path on disk within the temp directory
        temp_path = os.path.join(tmp_dir, artifact_name)
        # Save then upload to W&B
        df.to_csv(temp_path,index=False)
        artifact = wandb.Artifact(
            name=artifact_name,
            type="raw_data",
            description=f"preprocessed data of dataset mlops-final-project/raw_data.csv:latest",
        )
        artifact.add_file(temp_path)
        run.log_artifact(artifact)
        # This waits for the artifact to be uploaded to W&B. If you
        # do not add this, the temp directory might be removed before
        # W&B had a chance to upload the datasets, and the upload
        # might fail
        artifact.wait()

In [33]:
run.finish()